In [ ]:
from requests import get
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from time import time
from IPython.display import clear_output
import pandas as pd
# Modification liée au changement de structure de l'url de recherche
# L'url n'est plus classée par page mais par premier élément
# Chaque page de recherche contient toujours 50 films
# Donc voici le nouvel url de recherche avec first_film qui prendra les valeurs 1, 51, 101,...:
# 'http://www.imdb.com/search/title?release_date='+ year_url +'&sort=num_votes,desc&start='+ start
 

In [ ]:
starts = [str(i) for i in range(1,201,50)]
years_url = [str(i) for i in range(2000,2020)]

In [ ]:
# 1. Redéclarer les listes
names = []
years = []
imdb_ratings =[]
metascores = []
votes = []

In [ ]:
# 2. préparer l'affichage des boucles
start_time = time()
requests = 0

In [ ]:
# 3. Pour chaque année de l'intervalle 2000-2017
for year_url in years_url:
    
    # 4. boucle sur starts contenant les premiers films des pages de 1 à 4
    for start in starts:
        
        # 5. Faire une requête GET
        response = get('http://www.imdb.com/search/title?release_date='+ year_url +'&sort=num_votes,desc&start='+ start)
        
        # 6. Pause la boucle de 8 à 15 secondes
        sleep(randint(8,15))
        
        # 7. Afficher les informations sur les requêtes
        requests += 1
        elapsed_time = time() - start_time
        print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)
        
        # 8. Avertir si le code status est différent de 200
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))
            
        # 9. Stopper la boucle si le nombre de requêtes est supérieur à 72
        if requests > 72:
            warn('Nombre de requêtes trop important')
            break
            
        # 10. Extraire le HTML avec BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')
        
        # 11. Sélectionner les 50 films de chaque page (container)
        mv_containers = page_html.find_all('div', class_="lister-item mode-advanced")
        
        # 12. Boucle pour chaque container
        for container in mv_containers:
            # 13. Si le film a un Metascore
            if container.find('div', class_='ratings-metascore') is not None:
                
                # 14. scrape le titre
                name = container.h3.a.text
                names.append(name)
                
                #15. scrape l'année
                year = container.h3.find('span', class_='lister-item-year').text
                years.append(year)
                
                #16. scrape la note IMDB
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)
                
                #17. scrape le Metascore
                metascore = container.find('span', class_='metascore').text
                metascores.append(int(metascore))
                
                #18. scrape le nombre de votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))
 


NameError: ignored

In [ ]:
# Afficher les 100 premiers films dans un DataFrame
movie_ratings = pd.DataFrame({
        'movie': names,
        'year': years,
        'imdb': imdb_ratings,
        'metascore': metascores,
        'votes': votes
    })
 
movie_ratings.head(100)


,movie,year,imdb,metascore,votes
0,Gladiator,(2000),8.5,67,1328111
1,Memento,(2000),8.4,80,1115662
2,Snatch,(2000),8.3,55,775685
3,Requiem for a Dream,(2000),8.3,68,759915
4,X-Men,(2000),7.4,64,567325
...,...,...,...,...,...
95,Get Carter,(2000),5.2,24,32493
96,Dracula 2000,(2000),4.9,26,31925
97,Dungeons & Dragons,(2000),3.6,14,31866
98,The Way of the Gun,(2000),6.7,49,31681


In [ ]:
print(movie_ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2932 entries, 0 to 2931
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movie      2932 non-null   object 
 1   year       2932 non-null   object 
 2   imdb       2932 non-null   float64
 3   metascore  2932 non-null   int64  
 4   votes      2932 non-null   int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 114.7+ KB
None


In [ ]:
import csv
#Export Pandas DataFrame to a CSV 
movie_ratings.to_csv(r'C:\Users\21698\Desktop\Work\movie_ratings.csv',sep=",",index = False, header=True)
